# ETL Process for retail data Warehouse

In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
import os

# Configurations
Since i'm downloading the data from the internet, I need to set the config for the download

In [ ]:
DATASET_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
DB_FILE = 'retail_dw.db'
SCHEMA_FILE = 'schema_design.sql'

# Start of ETL Pipeline

In [3]:
print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Starting EXTRACTION phase...")

[2025-08-12 16:41:32] Starting EXTRACTION phase...


# Extracting Data from UCI Online Retail Dataset

In [4]:
try:
    df = pd.read_excel(DATASET_URL)
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Successfully extracted {len(df)} rows.")
    print(f"Dataset shape: {df.shape}")
    print("\nFirst few rows:")
    display(df.head())
except Exception as e:
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Error during extraction: {e}")
    df = None


[2025-08-12 16:44:09] Successfully extracted 541909 rows.
Dataset shape: (541909, 8)

First few rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


# Transformation

### cleaning and Preprocessing

In [5]:
    print("\nOriginal data info:")
    print(f"Total rows: {len(df)}")
    print(f"Missing CustomerID: {df['CustomerID'].isna().sum()}")


Original data info:
Total rows: 541909
Missing CustomerID: 135080


### Dropping Rowns with Missing CustomerID

In [6]:
df.dropna(subset=['CustomerID'], inplace=True)
print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Rows after dropping NA CustomerID: {len(df)}")
    

[2025-08-12 16:46:13] Rows after dropping NA CustomerID: 406829


### Converting Customer ID to Integer

In [7]:
df['CustomerID'] = df['CustomerID'].astype(int)

### Removing 0 quantitu and 0 Unit Price

In [8]:
 df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

### Converting InvoiceDate to datetime Object

In [9]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


# Feature Engineering and Filtering


### Calculate Total Sales as a New Column

In [10]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

### Filter for the last year of data in the dataset

In [11]:
last_date = df['InvoiceDate'].max()
one_year_prior = last_date - pd.DateOffset(years=1)
df_filtered = df[df['InvoiceDate'] >= one_year_prior].copy()
print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Rows after filtering for the last year: {len(df_filtered)}")
print(f"Date range: {df_filtered['InvoiceDate'].min()} to {df_filtered['InvoiceDate'].max()}")


[2025-08-12 16:50:49] Rows after filtering for the last year: 384529
Date range: 2010-12-09 12:59:00 to 2011-12-09 12:50:00


### Product Categorization
Writing a Function to help categorize the products

In [12]:
def categorize_product(description):
    description = str(description).lower()
    if 'set' in description or 'kit' in description:
        return 'Kits'
    if 'bag' in description or 'box' in description:
        return 'Storage'
    if 'light' in description or 'lamp' in description:
        return 'Decor'
    if 'cake' in description or 'party' in description:
        return 'Party Supplies'
    return 'Other'

Implementing the function

In [13]:
df_filtered['Category'] = df_filtered['Description'].apply(categorize_product)

print("Product categories distribution:")
print(df_filtered['Category'].value_counts())

Product categories distribution:
Other             242838
Kits               54368
Storage            53864
Party Supplies     17834
Decor              15625
Name: Category, dtype: int64


### Creating a Time dimension

In [14]:
dim_time_df = df_filtered[['InvoiceDate']].copy()
dim_time_df.drop_duplicates(inplace=True)
dim_time_df['Day'] = dim_time_df['InvoiceDate'].dt.day
dim_time_df['Month'] = dim_time_df['InvoiceDate'].dt.month
dim_time_df['Quarter'] = dim_time_df['InvoiceDate'].dt.quarter
dim_time_df['Year'] = dim_time_df['InvoiceDate'].dt.year
dim_time_df.reset_index(drop=True, inplace=True)
dim_time_df['TimeID'] = dim_time_df.index + 1  # Create a surrogate key

print(f"Time dimension created with {len(dim_time_df)} unique dates")
print("\nTime dimension sample:")
display(dim_time_df.head())

Time dimension created with 16630 unique dates

Time dimension sample:


,InvoiceDate,Day,Month,Quarter,Year,TimeID
0,2010-12-09 12:59:00,9,12,4,2010,1
1,2010-12-09 13:03:00,9,12,4,2010,2
2,2010-12-09 13:05:00,9,12,4,2010,3
3,2010-12-09 13:08:00,9,12,4,2010,4
4,2010-12-09 13:14:00,9,12,4,2010,5


### Creating Customer Dimension

In [15]:
dim_customer_df = df_filtered[['CustomerID', 'Country']].copy()
dim_customer_df.drop_duplicates(subset=['CustomerID'], inplace=True)

print(f"Customer dimension created with {len(dim_customer_df)} unique customers")
print(f"Countries represented: {dim_customer_df['Country'].nunique()}")
print("\nTop countries by customer count:")
print(dim_customer_df['Country'].value_counts().head())

Customer dimension created with 4269 unique customers
Countries represented: 36

Top countries by customer count:
United Kingdom    3859
Germany             93
France              85
Spain               28
Belgium             24
Name: Country, dtype: int64


### Creating Product Dimension

In [16]:
dim_product_df = df_filtered[['StockCode', 'Description', 'Category']].copy()
dim_product_df.drop_duplicates(subset=['StockCode'], inplace=True)
dim_product_df.reset_index(drop=True, inplace=True)
dim_product_df['ProductID'] = dim_product_df.index + 1  # Create a surrogate key

print(f"Product dimension created with {len(dim_product_df)} unique products")
print("\nProduct dimension sample:")
display(dim_product_df.head())


Product dimension created with 3630 unique products

Product dimension sample:


,StockCode,Description,Category,ProductID
0,22669,RED BABY BUNTING,Other,1
1,22465,HANGING METAL STAR LANTERN,Other,2
2,85123A,WHITE HANGING HEART T-LIGHT HOLDER,Decor,3
3,22727,ALARM CLOCK BAKELIKE RED,Other,4
4,22726,ALARM CLOCK BAKELIKE GREEN,Other,5


### Creating Fact Table

In [17]:
fact_sales_df = df_filtered.merge(dim_time_df, on='InvoiceDate')
fact_sales_df = fact_sales_df.merge(dim_product_df, on=['StockCode', 'Description', 'Category'])
fact_sales_df = fact_sales_df.merge(dim_customer_df, on='CustomerID')

# Select and rename columns for the final fact table
fact_sales_df = fact_sales_df[['InvoiceNo', 'Quantity', 'UnitPrice', 'TotalSales', 'TimeID', 'ProductID', 'CustomerID']]

print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Transformation complete. Processed {len(fact_sales_df)} fact rows.")
print("\nFact table sample:")
display(fact_sales_df.head())

print(f"\nFact table statistics:")
print(f"Total sales amount: ${fact_sales_df['TotalSales'].sum():,.2f}")
print(f"Average order value: ${fact_sales_df['TotalSales'].mean():.2f}")


[2025-08-12 16:59:54] Transformation complete. Processed 371053 fact rows.

Fact table sample:


,InvoiceNo,Quantity,UnitPrice,TotalSales,TimeID,ProductID,CustomerID
0,538032,5,2.95,14.75,1,1,14479
1,538032,12,1.65,19.80,1,2,14479
2,538032,6,2.95,17.70,1,3,14479
3,538032,4,3.75,15.00,1,4,14479
4,538032,4,3.75,15.00,1,5,14479



Fact table statistics:
Total sales amount: $8,261,140.72
Average order value: $22.26


# Start Loading Phase

In [18]:
print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Starting LOAD phase...")

# Remove old DB file to ensure a fresh start
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Removed existing database file: {DB_FILE}")

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

[2025-08-12 17:00:45] Starting LOAD phase...
[2025-08-12 17:00:45] Removed existing database file: retail_dw.db


# Creating Database schema and loading data

In [19]:
if df is not None and 'fact_sales_df' in locals():
    try:
        # Create tables from schema file (if it exists)
        if os.path.exists(SCHEMA_FILE):
            with open(SCHEMA_FILE, 'r') as f:
                cursor.executescript(f.read())
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Database schema created successfully.")
        else:
            print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Warning: Schema file '{SCHEMA_FILE}' not found. Creating tables directly.")
        
        # Load data into tables
        dim_time_df.to_sql('DimTime', conn, if_exists='replace', index=False)
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loaded {len(dim_time_df)} rows into DimTime.")
        
        dim_product_df.to_sql('DimProduct', conn, if_exists='replace', index=False)
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loaded {len(dim_product_df)} rows into DimProduct.")
        
        dim_customer_df.to_sql('DimCustomer', conn, if_exists='replace', index=False)
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loaded {len(dim_customer_df)} rows into DimCustomer.")
        
        fact_sales_df.to_sql('FactSales', conn, if_exists='replace', index=False)
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Loaded {len(fact_sales_df)} rows into FactSales.")
        
        conn.commit()
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Data loaded successfully into the warehouse.")
        
    except Exception as e:
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Error during load phase: {e}")
        conn.rollback()
    finally:
        conn.close()
        print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] Database connection closed.")


[2025-08-12 17:02:04] Warning: Schema file 'schema_design.sql' not found. Creating tables directly.
[2025-08-12 17:02:04] Loaded 16630 rows into DimTime.
[2025-08-12 17:02:04] Loaded 3630 rows into DimProduct.
[2025-08-12 17:02:04] Loaded 4269 rows into DimCustomer.
[2025-08-12 17:02:05] Loaded 371053 rows into FactSales.
[2025-08-12 17:02:05] Data loaded successfully into the warehouse.
[2025-08-12 17:02:05] Database connection closed.


### Verifying loaded Data

In [20]:
if os.path.exists(DB_FILE):
    conn = sqlite3.connect(DB_FILE)
    
    # Check table counts
    tables = ['DimTime', 'DimProduct', 'DimCustomer', 'FactSales']
    for table in tables:
        try:
            count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table}", conn).iloc[0]['count']
            print(f"{table}: {count} rows")
        except Exception as e:
            print(f"{table}: Error - {e}")
    
    conn.close()
else:
    print("Database file not found.")

DimTime: 16630 rows
DimProduct: 3630 rows
DimCustomer: 4269 rows
FactSales: 371053 rows


### Pipeline complete